# API-Ablauf Python Beispiel

Laden Sie dieses Jupyter-Notebook als [.ipynb 💾](/assets/downloads/mrt-api.ipynb) oder [.html 💾](/assets/downloads/mrt-api.ipynb)herunter.

## Vorbereitung

In [1]:
import requests
import json

# Vorbereitung für REST-Requests

# Session
hostname = "https://tour.multiroute.de/" # für SaaS; für InHouse entsprechend anpassen "https://tour.multiroute.de/"
session = requests.Session()

# Authentifizierung
session.auth = ("USERNAME", "PASSWORD") # Credentials
auth = session.post(hostname)

# Header
headers_mrt = {'Content-type': 'application/json'}


## 1. Adressen hochladen 
http://domain/fernsteuerung/add_and_update_adresse

In [2]:
# Bsp. data mit Koordinaten

data = {
"update": True,
"insert": True, 
"adresse": [{
    "oi": "GBC93385",
    "plz": "10119",
    "ort": "Berlin",
    "ort_zusatz": None,
    "strasse": "Schwedter Straße",
    "hausnummer": "22",
    "hausnummer_zusatz": "",
    "is_active": True,
    "notice": None,
    "privathaushalte": 0,
    "gewerbebetriebe": 0,
    "werbeverweigerer": 0,
    "longitude": 52.5347133208835,
    "latitude": 13.4075689315796
}, {
    "oi": "GBC360693",
    "plz": "84030",
    "ort": "Ergolding",
    "ort_zusatz": None,
    "strasse": "Schinderstraßl",
    "hausnummer": "5",
    "hausnummer_zusatz": "D",
    "is_active": True,
    "notice": None,
    "privathaushalte": 0,
    "gewerbebetriebe": 0,
    "werbeverweigerer": None,
    "longitude": 12.1660333871841,
    "latitude": 48.5649005287186
}]
 }

In [142]:
# Bsp. data ohne Koordinaten

data = {
"update": True,
"insert": True, 
"adresse": [{
    "oi": None,
    "plz": "10119",
    "ort": "Berlin",
    "ort_zusatz": None,
    "strasse": "Schwedter Straße",
    "hausnummer": "22",
    "hausnummer_zusatz": "",
    "is_active": True,
    "notice": None,
    "privathaushalte": 0,
    "gewerbebetriebe": 0,
    "werbeverweigerer": 0
    # entweder lat long ganz weglassen ...
}, {
    "oi": None,
    "plz": "84030",
    "ort": "Ergolding",
    "ort_zusatz": None,
    "strasse": "Schinderstraßl",
    "hausnummer": "5",
    "hausnummer_zusatz": "D",
    "is_active": True,
    "notice": None,
    "privathaushalte": 0,
    "gewerbebetriebe": 0,
    "werbeverweigerer": None,
    # ... oder None angeben - beides funktioniert.
    "longitude": None,
    "latitude": None
}]
 }

In [3]:
res = session.post(hostname + 'fernsteuerung/add_and_update_adresse/', headers=headers_mrt, data=json.dumps(data))
obj = json.loads(res.text)
upload_id = obj["upload_id"]
obj

{'upload_id': 587699,
 'text': '2 Adressen hinzugefügt, 0 Adressen geupdated ',
 'added': 2,
 'updated': 0}

## 2. Geokodierung der Adressen prüfen 

Nur nötig wenn Adressen ohne Koordinaten hochgeladen wurden. 

<http://domain/fernsteuerung/check_geocode?upload_id=upload_id>

In [4]:
res = session.get('{}fernsteuerung/check_geocode?upload_id={}'.format(hostname, upload_id))
obj = json.loads(res.text)
obj

{'upload_id': 587699, 'status': None, 'geocoded': 2, 'not_geocoded': 0}

## 3. Planung anlegen
http://domain/fernsteuerung/planung/add_and_update

In [5]:
# Es können auch mehrere Planungen auf einmal angelegt werden. Hier für zwei Werktage.

data = {"update": True,
"insert": True, 
"planung": [{
    "nr": "Montag, 04.10.2021",
    "name": "",
    "depot": None 
}, {
    "nr": "Dienstag, 05.10.2021",
    "name": "",
    "depot": None}] }

res = session.post(hostname + 'fernsteuerung/planung/add_and_update/', headers=headers_mrt, data=json.dumps(data))
obj = json.loads(res.text)
obj

{'text': '2 Planungen hinzugefügt, 0 Planungen geupdated ',
 'added': 2,
 'updated': 0}

## 4. Aufträge an Adresse spielen und Planung zuweisen
http://domain/fernsteuerung/auftrag

In [6]:
data = {
  "auftraege": [{
    "adresse_oid": "GBC360693",
    "planung_nr": "Montag, 04.10.2021",
    "name": "Musterauftrag, Kunde XYZ",
    "timewindows": "[08:00,12:00];[14:00,18:00]",
    "capacity": "2000" 
  }]
}

# mehr Aufträge hinzufügen!

res = session.post(hostname + 'fernsteuerung/auftrag/', headers=headers_mrt, data=json.dumps(data))
obj = json.loads(res.text)
obj

{'status': 200,
 'text': 'Aufträge erfolgreich angelegt.',
 'auftraege': [{'ext_id': None, 'id': 1391922011}]}

## 5. Flotte anlegen
http://domain/fernsteuerung/flotte

In [18]:
# Flotte mit 4 Fahrzeugen: 2 Sprinter, 1 PKW, 1 Kühlfahrzeug mit entsprechendem Skill

data = {
  "fleet": {
    "name": "Standardflotte",
    "vehicles": [
      {
        "name": "Sprinter",
        "number": 2,
        "amount": 2,
        "start": {
          "type": "start",
          "oid": "GBC31894593"
        },
        "end": {
          "type": "end",
          "oid": "GBC31894593"
        },
        "capacity": 1800000,
        "driving": "[08:00,12:00]",
        "break": {
          "timewindows": "[]",
          "duration": 0
        },
        "skills": [""]
      },
      {
        "name": "PKW",
        "number": 3,
        "amount": 2,
        "start": {
          "type": "start",
          "oid": "GBC31894593"
        },
        "end": {
          "type": "end",
          "oid": "GBC31894593"
        },
        "capacity": 800000,
        "driving": "[08:00,12:00]",
        "break": {
          "timewindows": "[]",
          "duration": 0
        },
        "skills": [""]
      },
      {
        "name": "Kühlfahrzeug",
        "number": 1,
        "amount": 1,
        "start": {
          "type": "start",
          "oid": "GBC31894593"
        },
        "end": {
          "type": "end",
          "oid": "GBC31894593"
        },
        "capacity": 1200000,
        "driving": "[08:00,12:00]",
        "break": {
          "timewindows": "[]",
          "duration": 0
        },
        "skills": [
          "Kühlfahrzeug"
        ]
      }
    ]
  }
}

res = session.post(hostname + 'fernsteuerung/flotte/', headers=headers_mrt, data=json.dumps(data))
obj = json.loads(res.text)
obj

{'status': 200, 'fleet_id': 3525}

## 6. Tourberechnung starten
http://domain/fernsteuerung/tour/berechnen

In [26]:
data = {
  "planung": "Montag, 04.10.2021",
  "fleet_name": "Standardflotte", # oder fleet_id, hier: 2913
  "tour": {
    "zeit_distanz": "zeit",
    "flat_deploy_time": 5.0,
    "deploy_time": 0.1,
    "maximum_jobs_per_vehicle": 100,
    "ignore_capacity_constraints": False,
    "zeit_distanz": "zeit"
  }
}

res = session.post(hostname + 'fernsteuerung/tour/berechnen', headers=headers_mrt, data=json.dumps(data))
obj = json.loads(res.text)
obj

{'status': 200, 'tour_id': 11735}

## 7. Prüfen ob Tourberechnung fertig & abfragen
<http://domain/fernsteuerung/touren?limit=1&planung=Nr_der_Planung>

In [27]:
# Key "success" prüfen ob bereits fertig berechnet. Falls nein, folgt der Output diesem Schema.

res = session.get(hostname + 'fernsteuerung/tour/' + "11735", headers=headers_mrt)
obj = json.loads(res.text)
obj

{'touren': [],
 'meta': {'id': 11735,
  'datenstand_osm': None,
  'success': None,
  'anz_fahrzeuge': None,
  'abladestellen': None,
  'error_message': '<p>Tour 11735 wird gerade berechnet.</p><p>Bitte haben Sie noch etwas Geduld.</p>',
  'horizont': None,
  'deploy_time': 0.1,
  'flat_deploy_time': '5.0',
  'max_deliver_time': None,
  'unassigned_jobs': None,
  'planung': 'Montag, 04.10.2021',
  'fleet': 'Standardflotte',
  'fleet_id': 3525,
  'zeit_distanz': 'Zeit',
  'gewicht': 0.0,
  'gewicht_text': 'kg',
  'strecke': 0.0,
  'dauer': 0.0,
  'wartezeit': 0.0,
  'entladedauer': 0.0,
  'gesamtdauer': 0.0,
  'berechnet_am': '08.07.2021 09:15',
  'depot_departure_time': None}}

In [42]:
# Wenn fertig, ganzes Tourenobjekt genauso abfragen.
res = session.get(hostname + 'fernsteuerung/tour/' + "11735", headers=headers_mrt)
obj = json.loads(res.text)
obj

```JSON
{
   "touren":[
      {
         "vehicle_name":"Sprinter",
         "vehicle_id":34056,
         "start_time":"08:00",
         "finish_time":"11:50",
         "open_end":false,
         "destination":true,
         "fahrzeugstrecke_id":545661,
         "nr":1,
         "index":0,
         "strecke":239046,
         "dauer":13562,
         "wartezeit":0,
         "entladedauer":275.6,
         "gesamtdauer":13837.6,
         "gewicht":1556.0,
         "gewicht_text":"kg",
         "fahrzeug_gewicht":1800.0,
         "fahrzeug_gewicht_text":"kg",
         "abladestellen":[
            {
               "sort":1,
               "oi":"457115A59778559",
               "plz":"83075",
               "ort":"Bad Feilnbach",
               "strasse":"Bahnhofstraße",
               "hausnummer":"5",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.77366,
               "lng":12.00978,
               "gewicht":95.0,
               "gewicht_text":"kg",
               "strecke":85642,
               "dauer":3456,
               "name":"Auftragsnummer KY-1261",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"08:57",
               "action_time":"08:57",
               "departure_time":"08:57",
               "service_sum":"0h 0\\' 14\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 9\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":2,
               "oi":"D8A7045409A8924",
               "plz":"83064",
               "ort":"Raubling",
               "strasse":"Tegernseestraße",
               "hausnummer":"1",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.76479,
               "lng":12.07856,
               "gewicht":30.0,
               "gewicht_text":"kg",
               "strecke":5842,
               "dauer":405,
               "name":"Auftragsnummer KY-1260",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:04",
               "action_time":"09:04",
               "departure_time":"09:04",
               "service_sum":"0h 0\\' 8\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 3\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":3,
               "oi":"2560CB9C2180745",
               "plz":"83064",
               "ort":"Raubling",
               "strasse":"Nicklheimer Straße",
               "hausnummer":"12",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.78706,
               "lng":12.10601,
               "gewicht":48.0,
               "gewicht_text":"kg",
               "strecke":4847,
               "dauer":335,
               "name":"Auftragsnummer KY-1262",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:10",
               "action_time":"09:10",
               "departure_time":"09:10",
               "service_sum":"0h 0\\' 9\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 4\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":4,
               "oi":"27369718B4F97A0",
               "plz":"83064",
               "ort":"Raubling",
               "strasse":"Holzbreitenweg",
               "hausnummer":"2",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.79101,
               "lng":12.11861,
               "gewicht":83.0,
               "gewicht_text":"kg",
               "strecke":1708,
               "dauer":197,
               "name":"Auftragsnummer KY-1263",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:13",
               "action_time":"09:13",
               "departure_time":"09:13",
               "service_sum":"0h 0\\' 13\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 8\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":5,
               "oi":"66C4A79097BFA12",
               "plz":"83101",
               "ort":"Rohrdorf",
               "strasse":"Hartseestraße",
               "hausnummer":"33",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.80995,
               "lng":12.19596,
               "gewicht":61.0,
               "gewicht_text":"kg",
               "strecke":9642,
               "dauer":665,
               "name":"Auftragsnummer KY-1256",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:25",
               "action_time":"09:25",
               "departure_time":"09:25",
               "service_sum":"0h 0\\' 11\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 6\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":6,
               "oi":"82B064585C81F5C",
               "plz":"83083",
               "ort":"Riedering",
               "strasse":"Söllhubener Straße",
               "hausnummer":"3",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.8378,
               "lng":12.20838,
               "gewicht":94.0,
               "gewicht_text":"kg",
               "strecke":4295,
               "dauer":286,
               "name":"Auftragsnummer KY-1255",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:30",
               "action_time":"09:30",
               "departure_time":"09:30",
               "service_sum":"0h 0\\' 14\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 9\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":7,
               "oi":"D064A0274A658B6",
               "plz":"83134",
               "ort":"Prutting",
               "strasse":"Rosenstraße",
               "hausnummer":"10",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.88919,
               "lng":12.15199,
               "gewicht":82.0,
               "gewicht_text":"kg",
               "strecke":8593,
               "dauer":625,
               "name":"Auftragsnummer KY-1271",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:40",
               "action_time":"09:40",
               "departure_time":"09:40",
               "service_sum":"0h 0\\' 13\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 8\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":8,
               "oi":"71F6C474ACD1089",
               "plz":"83071",
               "ort":"Stephanskirchen",
               "strasse":"Hofaustraße",
               "hausnummer":"19",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.86332,
               "lng":12.13875,
               "gewicht":64.0,
               "gewicht_text":"kg",
               "strecke":5001,
               "dauer":448,
               "name":"Auftragsnummer KY-1272",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:48",
               "action_time":"09:48",
               "departure_time":"09:48",
               "service_sum":"0h 0\\' 11\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 6\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":9,
               "oi":"5F6F4B9ECEAA3E6",
               "plz":"83022",
               "ort":"Rosenheim",
               "strasse":"Ellmaierstraße",
               "hausnummer":"3",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.85885,
               "lng":12.12894,
               "gewicht":52.0,
               "gewicht_text":"kg",
               "strecke":2394,
               "dauer":241,
               "name":"Auftragsnummer KY-1270",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:52",
               "action_time":"09:52",
               "departure_time":"09:52",
               "service_sum":"0h 0\\' 10\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 5\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":10,
               "oi":"F68DDAA77E48EA4",
               "plz":"83022",
               "ort":"Rosenheim",
               "strasse":"Steinbökstraße",
               "hausnummer":"2",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.85793,
               "lng":12.12676,
               "gewicht":90.0,
               "gewicht_text":"kg",
               "strecke":292,
               "dauer":34,
               "name":"Auftragsnummer KY-1269",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:53",
               "action_time":"09:53",
               "departure_time":"09:53",
               "service_sum":"0h 0\\' 14\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 9\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":11,
               "oi":"2FEF99C26DD92B5",
               "plz":"83022",
               "ort":"Rosenheim",
               "strasse":"Stollstraße",
               "hausnummer":"18",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.85724,
               "lng":12.12664,
               "gewicht":88.0,
               "gewicht_text":"kg",
               "strecke":83,
               "dauer":8,
               "name":"Auftragsnummer KY-1268",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:53",
               "action_time":"09:53",
               "departure_time":"09:53",
               "service_sum":"0h 0\\' 13\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 8\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":12,
               "oi":"3010ACFCC7BE97F",
               "plz":"83024",
               "ort":"Rosenheim",
               "strasse":"Carl-Orff-Straße",
               "hausnummer":"8",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.86952,
               "lng":12.13063,
               "gewicht":79.0,
               "gewicht_text":"kg",
               "strecke":1729,
               "dauer":208,
               "name":"Auftragsnummer KY-1273",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:57",
               "action_time":"09:57",
               "departure_time":"09:57",
               "service_sum":"0h 0\\' 12\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 7\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":13,
               "oi":"85F93A8E7A16B70",
               "plz":"83024",
               "ort":"Rosenheim",
               "strasse":"Ihlaustraße",
               "hausnummer":"12",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.87048,
               "lng":12.12333,
               "gewicht":69.0,
               "gewicht_text":"kg",
               "strecke":848,
               "dauer":153,
               "name":"Auftragsnummer KY-1274",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:00",
               "action_time":"10:00",
               "departure_time":"10:00",
               "service_sum":"0h 0\\' 11\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 6\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":14,
               "oi":"05029D4A2E858C5",
               "plz":"83024",
               "ort":"Rosenheim",
               "strasse":"Sepp-Sebald-Siedlung",
               "hausnummer":"39",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.86481,
               "lng":12.11373,
               "gewicht":50.0,
               "gewicht_text":"kg",
               "strecke":1306,
               "dauer":206,
               "name":"Auftragsnummer KY-1275",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:03",
               "action_time":"10:03",
               "departure_time":"10:03",
               "service_sum":"0h 0\\' 10\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 5\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":15,
               "oi":"9AF0873F7501832",
               "plz":"83024",
               "ort":"Rosenheim",
               "strasse":"Marienberger Straße",
               "hausnummer":"44",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.86867,
               "lng":12.10794,
               "gewicht":52.0,
               "gewicht_text":"kg",
               "strecke":1119,
               "dauer":183,
               "name":"Auftragsnummer KY-1276",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:07",
               "action_time":"10:07",
               "departure_time":"10:07",
               "service_sum":"0h 0\\' 10\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 5\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":16,
               "oi":"FD215F042BD79C1",
               "plz":"83026",
               "ort":"Rosenheim",
               "strasse":"Oberwöhrstraße",
               "hausnummer":"1",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.84814,
               "lng":12.10563,
               "gewicht":76.0,
               "gewicht_text":"kg",
               "strecke":2797,
               "dauer":399,
               "name":"Auftragsnummer KY-1277",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:13",
               "action_time":"10:13",
               "departure_time":"10:14",
               "service_sum":"0h 0\\' 12\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 7\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":17,
               "oi":"F70D2DC512B4A7F",
               "plz":"83026",
               "ort":"Rosenheim",
               "strasse":"Kufsteiner Straße",
               "hausnummer":"46",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.849911,
               "lng":12.127841,
               "gewicht":119.0,
               "gewicht_text":"kg",
               "strecke":2449,
               "dauer":280,
               "name":"Auftragsnummer KY-1267",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:18",
               "action_time":"10:18",
               "departure_time":"10:18",
               "service_sum":"0h 0\\' 16\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 11\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":18,
               "oi":"31B4791D319B268",
               "plz":"83022",
               "ort":"Rosenheim",
               "strasse":"Leitzachstraße",
               "hausnummer":"3",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.84553,
               "lng":12.13155,
               "gewicht":43.0,
               "gewicht_text":"kg",
               "strecke":1614,
               "dauer":213,
               "name":"Auftragsnummer KY-1266",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:22",
               "action_time":"10:22",
               "departure_time":"10:22",
               "service_sum":"0h 0\\' 9\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 4\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":19,
               "oi":"3808C738D1280AA",
               "plz":"83026",
               "ort":"Rosenheim",
               "strasse":"Asamstraße",
               "hausnummer":"32",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.83006,
               "lng":12.11518,
               "gewicht":38.0,
               "gewicht_text":"kg",
               "strecke":3130,
               "dauer":368,
               "name":"Auftragsnummer KY-1264",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:28",
               "action_time":"10:28",
               "departure_time":"10:28",
               "service_sum":"0h 0\\' 8\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 3\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":20,
               "oi":"7733127CECEF593",
               "plz":"83026",
               "ort":"Rosenheim",
               "strasse":"Sulzbergstraße",
               "hausnummer":"7",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.83102,
               "lng":12.10355,
               "gewicht":78.0,
               "gewicht_text":"kg",
               "strecke":1157,
               "dauer":205,
               "name":"Auftragsnummer KY-1265",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:32",
               "action_time":"10:32",
               "departure_time":"10:32",
               "service_sum":"0h 0\\' 12\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 7\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":21,
               "oi":"AB634CC94975D54",
               "plz":"83026",
               "ort":"Rosenheim",
               "strasse":"Gerhardingerweg",
               "hausnummer":"12",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.84199,
               "lng":12.09988,
               "gewicht":64.0,
               "gewicht_text":"kg",
               "strecke":3291,
               "dauer":461,
               "name":"Auftragsnummer KY-1278",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:40",
               "action_time":"10:40",
               "departure_time":"10:40",
               "service_sum":"0h 0\\' 11\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 6\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":22,
               "oi":"86EF8C350A3A556",
               "plz":"83059",
               "ort":"Kolbermoor",
               "strasse":"Wendelsteinstraße",
               "hausnummer":"14",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.84584,
               "lng":12.06875,
               "gewicht":32.0,
               "gewicht_text":"kg",
               "strecke":3650,
               "dauer":427,
               "name":"Auftragsnummer KY-1281",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:47",
               "action_time":"10:47",
               "departure_time":"10:47",
               "service_sum":"0h 0\\' 8\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 3\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":23,
               "oi":"D58BB86F715A268",
               "plz":"83059",
               "ort":"Kolbermoor",
               "strasse":"Farrenpointstraße",
               "hausnummer":"19",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.84551,
               "lng":12.05548,
               "gewicht":68.0,
               "gewicht_text":"kg",
               "strecke":1622,
               "dauer":201,
               "name":"Auftragsnummer KY-1280",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:51",
               "action_time":"10:51",
               "departure_time":"10:51",
               "service_sum":"0h 0\\' 11\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 6\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":24,
               "oi":"7E62734596587C8",
               "plz":"83059",
               "ort":"Kolbermoor",
               "strasse":"Aiblinger Au",
               "hausnummer":"14",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.84103,
               "lng":12.03726,
               "gewicht":1.0,
               "gewicht_text":"kg",
               "strecke":1652,
               "dauer":174,
               "name":"Auftragsnummer KY-1279",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:54",
               "action_time":"10:54",
               "departure_time":"10:54",
               "service_sum":"0h 0\\' 5\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 0\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":25,
               "oi":"GBC31894593",
               "plz":"85764",
               "ort":"Oberschleißheim",
               "strasse":"Mittenheimer Straße",
               "hausnummer":"58",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":48.2600842583442,
               "lng":11.5591406822205,
               "gewicht":0.0,
               "gewicht_text":"kg",
               "strecke":84343,
               "dauer":3384,
               "name":"Endpunkt",
               "description":"None",
               "timewindows":"None",
               "arrival_time":"11:50",
               "action_time":"None",
               "departure_time":"None",
               "service_sum":"None",
               "service_base":"None",
               "service_unit":"None",
               "service_indi":"None",
               "tour_break":false,
               "destination":true
            }
         ],
         "rueckweg_zum_depot":{
            "strecke":84343,
            "dauer":3384
         },
         "token":"ADC4A4AE6EF70D559C7C5DA6160B605B30F542219E694D555A0D9FA93F1E40D019CBC2644B0C86508F18484D12D85FA66F4CB44B158506EF84F591E5ED5CB678",
         "google_maps_url":"https://zustellplaner.biz/google_maps?token=ADC4A4AE6EF70D559C7C5DA6160B605B30F542219E694D555A0D9FA93F1E40D019CBC2644B0C86508F18484D12D85FA66F4CB44B158506EF84F591E5ED5CB678",
         "apple_maps_url":"https://zustellplaner.biz/apple_maps?token=ADC4A4AE6EF70D559C7C5DA6160B605B30F542219E694D555A0D9FA93F1E40D019CBC2644B0C86508F18484D12D85FA66F4CB44B158506EF84F591E5ED5CB678"
      },
      {
         "vehicle_name":"Sprinter",
         "vehicle_id":34056,
         "start_time":"08:00",
         "finish_time":"11:55",
         "open_end":false,
         "destination":true,
         "fahrzeugstrecke_id":545660,
         "nr":2,
         "index":1,
         "strecke":264584,
         "dauer":13981,
         "wartezeit":0,
         "entladedauer":167.0,
         "gesamtdauer":14148.0,
         "gewicht":1070.0,
         "gewicht_text":"kg",
         "fahrzeug_gewicht":1800.0,
         "fahrzeug_gewicht_text":"kg",
         "abladestellen":[
            {
               "sort":1,
               "oi":"E42A79E9F7BC1EB",
               "plz":"83080",
               "ort":"Oberaudorf",
               "strasse":"Naunspitzstraße",
               "hausnummer":"14",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.64805,
               "lng":12.17961,
               "gewicht":122.0,
               "gewicht_text":"kg",
               "strecke":106997,
               "dauer":4257,
               "name":"Auftragsnummer KY-1258",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:10",
               "action_time":"09:10",
               "departure_time":"09:11",
               "service_sum":"0h 0\\' 17\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 12\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":2,
               "oi":"74204ECEA69F87B",
               "plz":"83088",
               "ort":"Kiefersfelden",
               "strasse":"Schusterweg",
               "hausnummer":"9",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.63683,
               "lng":12.17445,
               "gewicht":105.0,
               "gewicht_text":"kg",
               "strecke":1931,
               "dauer":201,
               "name":"Auftragsnummer KY-1259",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:14",
               "action_time":"09:14",
               "departure_time":"09:14",
               "service_sum":"0h 0\\' 15\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 10\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":3,
               "oi":"E35E1C0172CC6FE",
               "plz":"83122",
               "ort":"Samerberg",
               "strasse":"Weikersing",
               "hausnummer":"11",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.7769,
               "lng":12.20319,
               "gewicht":53.0,
               "gewicht_text":"kg",
               "strecke":25160,
               "dauer":1400,
               "name":"Auftragsnummer KY-1257",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:38",
               "action_time":"09:38",
               "departure_time":"09:38",
               "service_sum":"0h 0\\' 10\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 5\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":4,
               "oi":"606DF5961392B7F",
               "plz":"83112",
               "ort":"Frasdorf",
               "strasse":"Dösdorf",
               "hausnummer":"74",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.81216,
               "lng":12.30798,
               "gewicht":99.0,
               "gewicht_text":"kg",
               "strecke":11317,
               "dauer":759,
               "name":"Auftragsnummer KY-1254",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:51",
               "action_time":"09:51",
               "departure_time":"09:51",
               "service_sum":"0h 0\\' 14\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 9\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":5,
               "oi":"6BDF2538BEDD1EB",
               "plz":"83209",
               "ort":"Prien am Chiemsee",
               "strasse":"Bernauer Straße",
               "hausnummer":"31",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.85064,
               "lng":12.34511,
               "gewicht":98.0,
               "gewicht_text":"kg",
               "strecke":7727,
               "dauer":554,
               "name":"Auftragsnummer KY-1253",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:00",
               "action_time":"10:00",
               "departure_time":"10:00",
               "service_sum":"0h 0\\' 14\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 9\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":6,
               "oi":"8DFC977010CA928",
               "plz":"83209",
               "ort":"Prien",
               "strasse":"Carl-Braun-Straße",
               "hausnummer":"1",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.862,
               "lng":12.35321,
               "gewicht":80.0,
               "gewicht_text":"kg",
               "strecke":1790,
               "dauer":214,
               "name":"Auftragsnummer KY-1252",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:04",
               "action_time":"10:04",
               "departure_time":"10:04",
               "service_sum":"0h 0\\' 13\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 8\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":7,
               "oi":"4D9085F436FD855",
               "plz":"83093",
               "ort":"Bad Endorf",
               "strasse":"Ströbinger Straße 18",
               "hausnummer":"52",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.90114,
               "lng":12.29928,
               "gewicht":103.0,
               "gewicht_text":"kg",
               "strecke":8464,
               "dauer":697,
               "name":"Auftragsnummer KY-1251",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:16",
               "action_time":"10:16",
               "departure_time":"10:16",
               "service_sum":"0h 0\\' 15\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 10\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":8,
               "oi":"FEC49D5C6AC3019",
               "plz":"83093",
               "ort":"Bad Endorf",
               "strasse":"Hartseestraße",
               "hausnummer":"3",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.91269,
               "lng":12.30679,
               "gewicht":122.0,
               "gewicht_text":"kg",
               "strecke":2113,
               "dauer":244,
               "name":"Auftragsnummer KY-1250",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:20",
               "action_time":"10:20",
               "departure_time":"10:20",
               "service_sum":"0h 0\\' 17\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 12\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":9,
               "oi":"05D31F77466335D",
               "plz":"83128",
               "ort":"Halfing",
               "strasse":"Holzham",
               "hausnummer":"12",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.95091,
               "lng":12.26076,
               "gewicht":34.0,
               "gewicht_text":"kg",
               "strecke":7314,
               "dauer":563,
               "name":"Auftragsnummer KY-1249",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:30",
               "action_time":"10:30",
               "departure_time":"10:30",
               "service_sum":"0h 0\\' 8\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 3\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":10,
               "oi":"8E2FAD86DCC8A9E",
               "plz":"83569",
               "ort":"Vogtareuth",
               "strasse":"Krankenhausstraße",
               "hausnummer":"20",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.94802,
               "lng":12.18768,
               "gewicht":100.0,
               "gewicht_text":"kg",
               "strecke":10273,
               "dauer":687,
               "name":"Auftragsnummer KY-1248",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:41",
               "action_time":"10:41",
               "departure_time":"10:42",
               "service_sum":"0h 0\\' 15\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 10\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":11,
               "oi":"CCBC05EFF609A94",
               "plz":"83512",
               "ort":"Wasserburg am Inn",
               "strasse":"Nordstraße",
               "hausnummer":"1",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":48.06153,
               "lng":12.18837,
               "gewicht":68.0,
               "gewicht_text":"kg",
               "strecke":15674,
               "dauer":960,
               "name":"Auftragsnummer KY-1247",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:58",
               "action_time":"10:58",
               "departure_time":"10:58",
               "service_sum":"0h 0\\' 11\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 6\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":12,
               "oi":"E74D47816589EE6",
               "plz":"83533",
               "ort":"Edling",
               "strasse":"Blumenstraße",
               "hausnummer":"30",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":48.05741,
               "lng":12.15873,
               "gewicht":86.0,
               "gewicht_text":"kg",
               "strecke":3031,
               "dauer":275,
               "name":"Auftragsnummer KY-1246",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"11:02",
               "action_time":"11:02",
               "departure_time":"11:02",
               "service_sum":"0h 0\\' 13\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 8\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":13,
               "oi":"GBC31894593",
               "plz":"85764",
               "ort":"Oberschleißheim",
               "strasse":"Mittenheimer Straße",
               "hausnummer":"58",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":48.2600842583442,
               "lng":11.5591406822205,
               "gewicht":0.0,
               "gewicht_text":"kg",
               "strecke":62793,
               "dauer":3170,
               "name":"Endpunkt",
               "description":"None",
               "timewindows":"None",
               "arrival_time":"11:55",
               "action_time":"None",
               "departure_time":"None",
               "service_sum":"None",
               "service_base":"None",
               "service_unit":"None",
               "service_indi":"None",
               "tour_break":false,
               "destination":true
            }
         ],
         "rueckweg_zum_depot":{
            "strecke":62793,
            "dauer":3170
         },
         "token":"ADC4A4AE6EF70D559C7C5DA6160B605B30F542219E694D555A0D9FA93F1E40D019CBC2644B0C86508F18484D12D85FA69A5FD38CEEA0C44F18385162AF83CE9A",
         "google_maps_url":"https://zustellplaner.biz/google_maps?token=ADC4A4AE6EF70D559C7C5DA6160B605B30F542219E694D555A0D9FA93F1E40D019CBC2644B0C86508F18484D12D85FA69A5FD38CEEA0C44F18385162AF83CE9A",
         "apple_maps_url":"https://zustellplaner.biz/apple_maps?token=ADC4A4AE6EF70D559C7C5DA6160B605B30F542219E694D555A0D9FA93F1E40D019CBC2644B0C86508F18484D12D85FA69A5FD38CEEA0C44F18385162AF83CE9A"
      },
      {
         "vehicle_name":"Kühlfahrzeug",
         "vehicle_id":34058,
         "start_time":"08:00",
         "finish_time":"11:11",
         "open_end":false,
         "destination":true,
         "fahrzeugstrecke_id":545659,
         "nr":3,
         "index":2,
         "strecke":181863,
         "dauer":11313,
         "wartezeit":0,
         "entladedauer":174.0,
         "gesamtdauer":11487.0,
         "gewicht":1040.0,
         "gewicht_text":"kg",
         "fahrzeug_gewicht":1200.0,
         "fahrzeug_gewicht_text":"kg",
         "abladestellen":[
            {
               "sort":1,
               "oi":"4DC53D65CE63633",
               "plz":"83620",
               "ort":"Feldkirchen-Westerham",
               "strasse":"Obereit",
               "hausnummer":"1",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.91227,
               "lng":11.82077,
               "gewicht":98.0,
               "gewicht_text":"kg",
               "strecke":56612,
               "dauer":2707,
               "name":"Auftragsnummer KY-1241",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"08:45",
               "action_time":"08:45",
               "departure_time":"08:45",
               "service_sum":"0h 0\\' 14\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 9\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":2,
               "oi":"9F7230C61575DBE",
               "plz":"83620",
               "ort":"Feldkirchen-Westerham",
               "strasse":"Jägerkampstraße",
               "hausnummer":"24",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.90467,
               "lng":11.83003,
               "gewicht":31.0,
               "gewicht_text":"kg",
               "strecke":3244,
               "dauer":288,
               "name":"Auftragsnummer KY-1240",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"08:50",
               "action_time":"08:50",
               "departure_time":"08:50",
               "service_sum":"0h 0\\' 8\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 3\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":3,
               "oi":"225AB984D518E40",
               "plz":"83052",
               "ort":"Bruckmühl",
               "strasse":"Rudolf-Harbig-Ring",
               "hausnummer":"60",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.87675,
               "lng":11.91883,
               "gewicht":49.0,
               "gewicht_text":"kg",
               "strecke":9350,
               "dauer":702,
               "name":"Auftragsnummer KY-1239",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:01",
               "action_time":"09:01",
               "departure_time":"09:02",
               "service_sum":"0h 0\\' 9\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 4\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":4,
               "oi":"77A1C236B56DE28",
               "plz":"83052",
               "ort":"Bruckmühl",
               "strasse":"Josef-Völkl-Straße",
               "hausnummer":"7",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.87035,
               "lng":11.9588,
               "gewicht":158.0,
               "gewicht_text":"kg",
               "strecke":5525,
               "dauer":499,
               "name":"Auftragsnummer KY-1238",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:10",
               "action_time":"09:10",
               "departure_time":"09:10",
               "service_sum":"0h 0\\' 20\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 15\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":5,
               "oi":"EDC0174325E42A4",
               "plz":"83043",
               "ort":"Bad Aibling",
               "strasse":"Gerhart-Hauptmann-Straße",
               "hausnummer":"19",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.86179,
               "lng":11.99362,
               "gewicht":77.0,
               "gewicht_text":"kg",
               "strecke":3529,
               "dauer":405,
               "name":"Auftragsnummer KY-1237",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:17",
               "action_time":"09:17",
               "departure_time":"09:17",
               "service_sum":"0h 0\\' 12\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 7\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":6,
               "oi":"AB8AE3B12BEF1E8",
               "plz":"83043",
               "ort":"Bad Aibling",
               "strasse":"Bahnhofstraße",
               "hausnummer":"1",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.86285,
               "lng":12.00767,
               "gewicht":5.0,
               "gewicht_text":"kg",
               "strecke":1380,
               "dauer":201,
               "name":"Auftragsnummer KY-1236",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:21",
               "action_time":"09:21",
               "departure_time":"09:21",
               "service_sum":"0h 0\\' 5\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 0\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":7,
               "oi":"21DFEAA43CFBAE0",
               "plz":"83043",
               "ort":"Bad Aibling",
               "strasse":"Kolpingstraße",
               "hausnummer":"3",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.86716,
               "lng":12.01467,
               "gewicht":38.0,
               "gewicht_text":"kg",
               "strecke":905,
               "dauer":143,
               "name":"Auftragsnummer KY-1235",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:23",
               "action_time":"09:23",
               "departure_time":"09:23",
               "service_sum":"0h 0\\' 8\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 3\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":8,
               "oi":"DBACFEF87B8A3BC",
               "plz":"83043",
               "ort":"Bad Aibling",
               "strasse":"Vogelweidstraße",
               "hausnummer":"41",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.87388,
               "lng":12.00917,
               "gewicht":117.0,
               "gewicht_text":"kg",
               "strecke":1200,
               "dauer":187,
               "name":"Auftragsnummer KY-1234",
               "description":"Tel. 0176-122345",
               "timewindows":"ohne",
               "arrival_time":"09:26",
               "action_time":"09:26",
               "departure_time":"09:27",
               "service_sum":"0h 0\\' 16\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 11\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":9,
               "oi":"2A594D0BAE6B9B6",
               "plz":"83109",
               "ort":"Großkarolinenfeld",
               "strasse":"Schultheiß-Gipp-Straße",
               "hausnummer":"23",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.88635,
               "lng":12.07052,
               "gewicht":121.0,
               "gewicht_text":"kg",
               "strecke":6549,
               "dauer":513,
               "name":"Frau Müller",
               "description":"Hofeinfahrt rechts",
               "timewindows":"ohne",
               "arrival_time":"09:35",
               "action_time":"09:35",
               "departure_time":"09:35",
               "service_sum":"0h 0\\' 17\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 12\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":10,
               "oi":"FAED9FAB50EED2D",
               "plz":"83109",
               "ort":"Großkarolinenfeld",
               "strasse":"Am Kolberg",
               "hausnummer":"12",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.8925,
               "lng":12.07356,
               "gewicht":92.0,
               "gewicht_text":"kg",
               "strecke":1642,
               "dauer":179,
               "name":"Auftragsnummer KY-1282",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:38",
               "action_time":"09:38",
               "departure_time":"09:39",
               "service_sum":"0h 0\\' 14\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 9\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":11,
               "oi":"171994E8F73936B",
               "plz":"83135",
               "ort":"Schechen",
               "strasse":"Friesing",
               "hausnummer":"14",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.94915,
               "lng":12.11905,
               "gewicht":29.0,
               "gewicht_text":"kg",
               "strecke":12385,
               "dauer":731,
               "name":"Auftragsnummer KY-1244",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:51",
               "action_time":"09:51",
               "departure_time":"09:51",
               "service_sum":"0h 0\\' 7\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 2\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":12,
               "oi":"23E252B2264F8FC",
               "plz":"83543",
               "ort":"Rott am Inn",
               "strasse":"Johannes-Held-Straße",
               "hausnummer":"9",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.99023,
               "lng":12.13081,
               "gewicht":61.0,
               "gewicht_text":"kg",
               "strecke":5432,
               "dauer":448,
               "name":"Auftragsnummer KY-1245",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"09:58",
               "action_time":"09:58",
               "departure_time":"09:59",
               "service_sum":"0h 0\\' 11\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 6\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":13,
               "oi":"37C1FD0BD733CB5",
               "plz":"83104",
               "ort":"Tuntenhausen",
               "strasse":"Lampferding",
               "hausnummer":"5",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":47.96042,
               "lng":12.05543,
               "gewicht":86.0,
               "gewicht_text":"kg",
               "strecke":7692,
               "dauer":540,
               "name":"Auftragsnummer KY-1243",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:08",
               "action_time":"10:08",
               "departure_time":"10:08",
               "service_sum":"0h 0\\' 13\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 8\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":14,
               "oi":"53631C65CE6B722",
               "plz":"85625",
               "ort":"Glonn",
               "strasse":"Schlacht",
               "hausnummer":"50",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":48.00311,
               "lng":11.84452,
               "gewicht":78.0,
               "gewicht_text":"kg",
               "strecke":23618,
               "dauer":1590,
               "name":"Auftragsnummer KY-1242",
               "description":"",
               "timewindows":"ohne",
               "arrival_time":"10:34",
               "action_time":"10:34",
               "departure_time":"10:35",
               "service_sum":"0h 0\\' 12\"",
               "service_base":"0h 0\\' 5\"",
               "service_unit":"0h 0\\' 7\"",
               "service_indi":"0h 0\\' 0\"",
               "tour_break":false,
               "destination":false
            },
            {
               "sort":15,
               "oi":"GBC31894593",
               "plz":"85764",
               "ort":"Oberschleißheim",
               "strasse":"Mittenheimer Straße",
               "hausnummer":"58",
               "hausnummer_zusatz":"",
               "notice":"None",
               "street_oi":"None",
               "lat":48.2600842583442,
               "lng":11.5591406822205,
               "gewicht":0.0,
               "gewicht_text":"kg",
               "strecke":42800,
               "dauer":2180,
               "name":"Endpunkt",
               "description":"None",
               "timewindows":"None",
               "arrival_time":"11:11",
               "action_time":"None",
               "departure_time":"None",
               "service_sum":"None",
               "service_base":"None",
               "service_unit":"None",
               "service_indi":"None",
               "tour_break":false,
               "destination":true
            }
         ],
         "rueckweg_zum_depot":{
            "strecke":42800,
            "dauer":2180
         },
         "token":"ADC4A4AE6EF70D559C7C5DA6160B605B30F542219E694D555A0D9FA93F1E40D019CBC2644B0C86508F18484D12D85FA6F380F5D4A7DFF2BAC5237F7CCB6DB441",
         "google_maps_url":"https://zustellplaner.biz/google_maps?token=ADC4A4AE6EF70D559C7C5DA6160B605B30F542219E694D555A0D9FA93F1E40D019CBC2644B0C86508F18484D12D85FA6F380F5D4A7DFF2BAC5237F7CCB6DB441",
         "apple_maps_url":"https://zustellplaner.biz/apple_maps?token=ADC4A4AE6EF70D559C7C5DA6160B605B30F542219E694D555A0D9FA93F1E40D019CBC2644B0C86508F18484D12D85FA6F380F5D4A7DFF2BAC5237F7CCB6DB441"
      }
   ],
   "meta":{
      "id":11735,
      "datenstand_osm":"13.06.2021",
      "success":true,
      "anz_fahrzeuge":3,
      "abladestellen":50,
      "error_message":"None",
      "horizont":"None",
      "deploy_time":0.1,
      "flat_deploy_time":"5.0",
      "max_deliver_time":"None",
      "unassigned_jobs":"1",
      "planung":"Montag, 04.10.2021",
      "fleet":"Standardflotte",
      "fleet_id":3525,
      "zeit_distanz":"Zeit",
      "gewicht":3666.0,
      "gewicht_text":"kg",
      "strecke":685493.0,
      "dauer":38856.0,
      "wartezeit":0.0,
      "entladedauer":593.0,
      "gesamtdauer":39449.0,
      "berechnet_am":"08.07.2021 09:15",
      "depot_departure_time":"08:00"
   },
   "depot":{
      "oi":"GBC31894593",
      "plz":"85764",
      "ort":"Oberschleißheim",
      "strasse":"Mittenheimer Straße",
      "hausnummer":"58",
      "hausnummer_zusatz":"",
      "notice":"None",
      "street_oi":"None",
      "departure_time":"08:00",
      "lat":48.2600842583442,
      "lng":11.5591406822205
   },
   "unassigned_jobs":[
      {
         "timewindows":"[08:00,12:00];[14:00,18:00]",
         "hausnummer":"5",
         "hausnummer_zusatz":"D",
         "strasse":"Schinderstraßl",
         "plz":"84030",
         "ort":"Ergolding",
         "oi":"GBC360693",
         "adresse_id":31894644,
         "adresse":"Schinderstraßl 5D, 84030 Ergolding",
         "name":"Musterauftrag, Kunde XYZ",
         "description":"None",
         "notice":"None",
         "service_sum":"0h 3\\' 25\"",
         "service_base":"0h 0\\' 5\"",
         "service_unit":"0h 3\\' 20\"",
         "service_indi":"0h 0\\' 0\"",
         "nr":1
      }
   ],
   "flotte":[
      {
         "anzahl":1,
         "gewicht":1200.0,
         "gewicht_text":"kg",
         "name":"Kühlfahrzeug",
         "number":1,
         "timewindow":"[08:00,12:00]",
         "start_time":"08:00",
         "end_time":"12:00"
      },
      {
         "anzahl":2,
         "gewicht":1800.0,
         "gewicht_text":"kg",
         "name":"Sprinter",
         "number":2,
         "timewindow":"[08:00,12:00]",
         "start_time":"08:00",
         "end_time":"12:00"
      }
   ],
   "diff_flotte":[
      {
         "anzahl":2,
         "gewicht":800.0,
         "gewicht_text":"kg",
         "name":"PKW",
         "number":3,
         "timewindow":"[08:00,12:00]",
         "start_time":"08:00",
         "end_time":"12:00"
      }
   ]
}
```

In [38]:
# Bessere JSON Visualisierung in Jupyter Notebooks.
from IPython.display import JSON
JSON(obj)